In [18]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import csv
import datetime as dt

In [19]:
# school
# hostip = "10.192.40.238"
# hostip = "10.192.13.255"

# home
# hostip = "192.168.0.88"
# hostip = "192.168.0.160"
# 
# hostip = "10.192.12.188"
# hostip = "10.192.40.238"

# hostip = "10.192.41.222"

hostip = "192.168.0.21"

topic = 'ProducerA'
# file_path = 'FIT3182-A2/data/camera_event_A.csv'
file_path = 'data/camera_event_A.csv'
# file_path = './A2/data/camera_event_A.csv' 




def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        print('Message published successfully. ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        

In [20]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [ ]:
if __name__ == '__main__':
    print('Publishing records..')
    producer = connect_kafka_producer()

    if producer is None:
        print('Failed to connect to Kafka. Exiting..')
        exit(1)

    current_batch = None
    batch_events = []

    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            json_row = dumps(row)
            if current_batch is None:
                current_batch = row['batch_id']

            if current_batch != row['batch_id']:
                for event in batch_events:
                    message = {
                        'event_id': event['event_id'],
                        "car_plate": event['car_plate'],
                        "camera_id": int(event['camera_id']),
                        "timestamp": event['timestamp'],
                        "speed_reading": float(event['speed_reading']),
                        "producer": "A"
                    }
                    publish_message(producer, topic, event['event_id'], dumps(message))
                producer.flush()
                print(f'Batch {current_batch} published successfully.')
                sleep(5)

                current_batch = row['batch_id']
                batch_events = []

            batch_events.append(row)

        if batch_events:
            for event in batch_events:
                message = {
                    'event_id': event['event_id'],
                    "car_plate": event['car_plate'],
                    "camera_id": int(event['camera_id']),
                    "timestamp": event['timestamp'],
                    "speed_reading": float(event['speed_reading']),
                    "producer": "A"
                }
                publish_message(producer, topic, event['event_id'], dumps(message))
            producer.flush()
            print(f'Final batch {current_batch} published successfully.')

    print('All messages published successfully.')

Publishing records..
Message published successfully. {"event_id": "d40c586c-5be6-4743-a1e3-2269d9edaa72", "car_plate": "KRN 7", "camera_id": 1, "timestamp": "2024-01-01T08:00:04", "speed_reading": 77.2, "producer": "A"}
Message published successfully. {"event_id": "85c08e3c-a0b5-45d8-a70c-df8f9a6d5829", "car_plate": "ICE 8", "camera_id": 1, "timestamp": "2024-01-01T08:00:05", "speed_reading": 103.7, "producer": "A"}
Message published successfully. {"event_id": "f5834b79-771b-4931-8da2-a5ad7f4ccd02", "car_plate": "QE 1820", "camera_id": 1, "timestamp": "2024-01-01T08:00:03", "speed_reading": 67.4, "producer": "A"}
Message published successfully. {"event_id": "d0e547bb-c4a7-4750-b7b4-8076e9b47f4f", "car_plate": "CJW 924", "camera_id": 1, "timestamp": "2024-01-01T08:00:01", "speed_reading": 148.3, "producer": "A"}
Message published successfully. {"event_id": "f3162606-1b2e-407f-951d-61d14c0a7b09", "car_plate": "CJP 278", "camera_id": 1, "timestamp": "2024-01-01T08:00:02", "speed_reading":

Message published successfully. {"event_id": "1dd988c0-d000-489b-8cf0-2cd738fb0565", "car_plate": "BQN 88", "camera_id": 1, "timestamp": "2024-01-01T08:19:48", "speed_reading": 61.3, "producer": "A"}
Message published successfully. {"event_id": "9478323b-a2f6-4592-8c9f-a9de5bb984ab", "car_plate": "ZEA 3530", "camera_id": 1, "timestamp": "2024-01-01T08:19:48", "speed_reading": 153.3, "producer": "A"}
Message published successfully. {"event_id": "3e613964-b717-44a8-bf81-a1290ae07a35", "car_plate": "SJ 15", "camera_id": 1, "timestamp": "2024-01-01T08:19:48", "speed_reading": 146.2, "producer": "A"}
Message published successfully. {"event_id": "9f538681-69f0-4efd-a3e3-ef44ee2c1094", "car_plate": "PB 55", "camera_id": 1, "timestamp": "2024-01-01T08:19:49", "speed_reading": 111.2, "producer": "A"}
Message published successfully. {"event_id": "26e1dd39-29ec-4692-bca0-5705579ed316", "car_plate": "BU 9", "camera_id": 1, "timestamp": "2024-01-01T08:19:47", "speed_reading": 93.4, "producer": "A"}

Message published successfully. {"event_id": "3b552227-5141-4478-ae2f-3290e69a10c9", "car_plate": "IMC 6788", "camera_id": 1, "timestamp": "2024-01-01T08:42:38", "speed_reading": 87.6, "producer": "A"}
Message published successfully. {"event_id": "b80e1666-e54c-408d-bd13-31907f32beb8", "car_plate": "UQ 07", "camera_id": 1, "timestamp": "2024-01-01T08:42:40", "speed_reading": 143.9, "producer": "A"}
Message published successfully. {"event_id": "d4a1869f-2472-4b79-99b9-aff566122d5a", "car_plate": "CXD 617", "camera_id": 1, "timestamp": "2024-01-01T08:42:41", "speed_reading": 75.0, "producer": "A"}
Message published successfully. {"event_id": "738b20ae-27b2-421f-aac6-7b85939e1371", "car_plate": "PJ 35", "camera_id": 1, "timestamp": "2024-01-01T08:42:39", "speed_reading": 115.9, "producer": "A"}
Message published successfully. {"event_id": "4224f70a-54a9-45b8-b2b2-6f79d2444837", "car_plate": "MC 2760", "camera_id": 1, "timestamp": "2024-01-01T08:42:41", "speed_reading": 70.6, "producer": "

Message published successfully. {"event_id": "0f2932d6-ea15-4ee9-b76c-5ddd45978e07", "car_plate": "WVU 913", "camera_id": 1, "timestamp": "2024-01-01T09:07:07", "speed_reading": 85.0, "producer": "A"}
Message published successfully. {"event_id": "f40f586f-63d3-4795-b858-26dc045d9b17", "car_plate": "HGQ 06", "camera_id": 1, "timestamp": "2024-01-01T09:07:06", "speed_reading": 87.4, "producer": "A"}
Message published successfully. {"event_id": "34991344-75d0-4a8a-8cf4-37970ff7afc5", "car_plate": "NA 632", "camera_id": 1, "timestamp": "2024-01-01T09:07:06", "speed_reading": 156.3, "producer": "A"}
Message published successfully. {"event_id": "9da727c5-c7e9-444b-884f-c7a0cfd2e859", "car_plate": "JR 506", "camera_id": 1, "timestamp": "2024-01-01T09:07:04", "speed_reading": 123.3, "producer": "A"}
Message published successfully. {"event_id": "ef6fe407-1d26-48fb-95a8-1868f05df0f3", "car_plate": "DN 1", "camera_id": 1, "timestamp": "2024-01-01T09:07:06", "speed_reading": 141.8, "producer": "A"

Message published successfully. {"event_id": "2e8b5c14-1d48-484c-8b3e-9500ee86717e", "car_plate": "HM 258", "camera_id": 1, "timestamp": "2024-01-01T09:31:22", "speed_reading": 60.2, "producer": "A"}
Message published successfully. {"event_id": "2b480838-5811-4479-b618-23f9d639eb22", "car_plate": "WG 9406", "camera_id": 1, "timestamp": "2024-01-01T09:31:22", "speed_reading": 146.0, "producer": "A"}
Message published successfully. {"event_id": "9472591a-9fef-4a05-8d77-8ce4eb57410b", "car_plate": "UWZ 88", "camera_id": 1, "timestamp": "2024-01-01T09:31:20", "speed_reading": 106.4, "producer": "A"}
Message published successfully. {"event_id": "5e764904-deaa-4d9c-871a-2da42a096726", "car_plate": "HZ 826", "camera_id": 1, "timestamp": "2024-01-01T09:31:21", "speed_reading": 68.4, "producer": "A"}
Message published successfully. {"event_id": "12247c1c-d06c-4c14-85d9-5ea85e3d86af", "car_plate": "GS 6268", "camera_id": 1, "timestamp": "2024-01-01T09:31:18", "speed_reading": 116.4, "producer": 

...

Message published successfully. {"event_id": "eee154f1-8d6a-4f42-a6ea-88b8fe3f77b3", "car_plate": "ZZS 15", "camera_id": 1, "timestamp": "2024-01-03T08:14:47", "speed_reading": 90.1, "producer": "A"}
Message published successfully. {"event_id": "b657b1ad-a4c6-48cc-93fa-b30d0a32dbce", "car_plate": "YBT 4065", "camera_id": 1, "timestamp": "2024-01-03T08:14:48", "speed_reading": 142.9, "producer": "A"}
Message published successfully. {"event_id": "f6da4d8d-8316-4ea4-8d1a-5341ab3f3f4e", "car_plate": "UH 1091", "camera_id": 1, "timestamp": "2024-01-03T08:14:47", "speed_reading": 137.0, "producer": "A"}
Message published successfully. {"event_id": "894bef2c-7c02-4cbb-ad10-45d5813d9a07", "car_plate": "HH 8", "camera_id": 1, "timestamp": "2024-01-03T08:14:48", "speed_reading": 130.3, "producer": "A"}
Message published successfully. {"event_id": "db092515-c610-41a7-819e-06b31f9ff00b", "car_plate": "VB 669", "camera_id": 1, "timestamp": "2024-01-03T08:14:44", "speed_reading": 100.3, "producer": 

Batch 186 published successfully.
Message published successfully. {"event_id": "8216222d-1406-45a4-838c-ec93247261d9", "car_plate": "MHC 0013", "camera_id": 1, "timestamp": "2024-01-03T08:37:04", "speed_reading": 103.5, "producer": "A"}
Message published successfully. {"event_id": "643162c7-3565-4cee-a85d-098439137828", "car_plate": "VAB 30", "camera_id": 1, "timestamp": "2024-01-03T08:37:01", "speed_reading": 122.2, "producer": "A"}
Message published successfully. {"event_id": "7b2250e0-06ad-491e-87f2-ea27be5047cd", "car_plate": "IWY 074", "camera_id": 1, "timestamp": "2024-01-03T08:37:03", "speed_reading": 73.3, "producer": "A"}
Message published successfully. {"event_id": "5e954654-41cf-4397-ab7a-c94696533a4e", "car_plate": "IQ 9873", "camera_id": 1, "timestamp": "2024-01-03T08:37:03", "speed_reading": 72.6, "producer": "A"}
Message published successfully. {"event_id": "401df8ae-400f-49b8-86f3-fca223d89d37", "car_plate": "WDU 212", "camera_id": 1, "timestamp": "2024-01-03T08:37:03",

Message published successfully. {"event_id": "563b41b2-b218-4a64-a26f-ebdd8c10253f", "car_plate": "BE 1", "camera_id": 1, "timestamp": "2024-01-03T09:00:26", "speed_reading": 84.8, "producer": "A"}
Message published successfully. {"event_id": "fbcd2a78-b07a-4d0a-9241-c98a8d2a8503", "car_plate": "CWY 82", "camera_id": 1, "timestamp": "2024-01-03T09:00:27", "speed_reading": 67.9, "producer": "A"}
Message published successfully. {"event_id": "0ad21df5-011a-47c8-a4fe-0cc1ae3f2875", "car_plate": "TP 986", "camera_id": 1, "timestamp": "2024-01-03T09:00:27", "speed_reading": 83.8, "producer": "A"}
Message published successfully. {"event_id": "b2345f1b-13c3-4c72-9772-15e546c99b51", "car_plate": "XUS 8375", "camera_id": 1, "timestamp": "2024-01-03T09:00:26", "speed_reading": 103.0, "producer": "A"}
Message published successfully. {"event_id": "74a05022-b3a4-4603-bc42-354fb50ca04e", "car_plate": "DH 438", "camera_id": 1, "timestamp": "2024-01-03T09:00:25", "speed_reading": 140.0, "producer": "A"

Message published successfully. {"event_id": "31350dcc-1ec4-43fc-b73a-214d5d94eca3", "car_plate": "FG 850", "camera_id": 1, "timestamp": "2024-01-03T09:21:56", "speed_reading": 140.5, "producer": "A"}
Message published successfully. {"event_id": "b64c4e17-99b5-4681-8185-0e367c64be9c", "car_plate": "ZMM 3274", "camera_id": 1, "timestamp": "2024-01-03T09:21:57", "speed_reading": 97.7, "producer": "A"}
Message published successfully. {"event_id": "b12d4502-abda-4f27-b803-37b86ecfa9ab", "car_plate": "NT 0", "camera_id": 1, "timestamp": "2024-01-03T09:21:54", "speed_reading": 109.8, "producer": "A"}
Message published successfully. {"event_id": "e50546b9-419f-4aab-89a4-bef4d5bb0ca3", "car_plate": "TF 3", "camera_id": 1, "timestamp": "2024-01-03T09:21:59", "speed_reading": 62.1, "producer": "A"}
Message published successfully. {"event_id": "63e25304-e94f-4262-bbd8-4fda103032cc", "car_plate": "UP 44", "camera_id": 1, "timestamp": "2024-01-03T09:21:55", "speed_reading": 84.8, "producer": "A"}
M

Message published successfully. {"event_id": "ca6b3a47-7c9f-4086-99e4-21ce8f2536e5", "car_plate": "WM 7013", "camera_id": 1, "timestamp": "2024-01-03T09:42:14", "speed_reading": 110.4, "producer": "A"}
Message published successfully. {"event_id": "9b04469a-bbcf-404c-aec3-2a7bf38b0fd0", "car_plate": "ZUA 719", "camera_id": 1, "timestamp": "2024-01-03T09:42:14", "speed_reading": 104.7, "producer": "A"}
Message published successfully. {"event_id": "afb85992-c9e1-41f7-9c75-3002b1f9b640", "car_plate": "AK 11", "camera_id": 1, "timestamp": "2024-01-03T09:42:13", "speed_reading": 71.4, "producer": "A"}
Message published successfully. {"event_id": "5028e726-7323-49d8-a1a4-863e1e32f2d3", "car_plate": "EMT 512", "camera_id": 1, "timestamp": "2024-01-03T09:42:13", "speed_reading": 85.3, "producer": "A"}
Message published successfully. {"event_id": "a94306b4-5938-475c-a8e1-70410393f96a", "car_plate": "VTM 97", "camera_id": 1, "timestamp": "2024-01-03T09:42:12", "speed_reading": 83.2, "producer": "